# PISA API demo
## Assemblies and interaction interfaces

The PDBe team introduced three new API endpoints, providing JSON-formatted responses. These endpoints retrieve data from PISA, focusing on protein interfaces and assemblies. You can find detailed JSON schemas for these endpoints on [Apiary](https://pisalite.docs.apiary.io/#reference/0/pisaqualifierjson/interaction-interface-data-per-pdb-assembly-entry), including comprehensive parameter descriptions for both assemblies and interfaces.


## 1. Details of an assembly

The specified API endpoint delivers JSON-formatted responses for a requested assembly based on the provided PDB entry (pdbid) and assembly ID (assemblyid). The JSON file includes parameters detailing the structural and thermodynamic properties of assemblies, such as the number of interfaces, macromolecular size, dissociation energy, and accessible surface area. You can access comprehensive parameter descriptions at [Apiary](https://pisalite.docs.apiary.io/#reference/0/pisaqualifierjson/interaction-interface-data-per-pdb-assembly-entry).

The API endpoint pattern is: *https://wwwdev.ebi.ac.uk/pdbe/api/pisa/assembly/:pdbid/:assemblyid*

For example, if you would like to send a request to get assembly details for entry **pdbid = 3gz1** and the preferred assembly is **assemblyid = 1**, the following Python script could help:

**Note:** You might first need to run `pip install requests` in your terminal


In [6]:
import requests 
response = requests.get("https://wwwdev.ebi.ac.uk/pdbe/api/pisa/assembly/3gz1/1")
response.json()

{'3gz1': {'assembly_id': '1',
  'pisa_version': '2.0',
  'assembly': {'id': '1',
   'size': '7',
   'interface_count': 3,
   'score': '',
   'macromolecular_size': '4',
   'dissociation_energy': 16.95,
   'accessible_surface_area': 15635.16,
   'buried_surface_area': 6031.94,
   'entropy': 19.65,
   'dissociation_area': 2034.9,
   'solvation_energy_gain': -35.09,
   'number_of_uc': '0',
   'number_of_dissociated_elements': '3',
   'symmetry_number': '1',
   'formula': 'A(2)BCa(3)',
   'composition': 'ABPQ[GOL](3)',
   'R350': ''}}}

You can extract specific information from this JSON response. For example, if you want to know the number of interfaces (`interface_count`) or the assembly dissociation energy  (`dissociation_energy`), the following script could help:


In [44]:
data = response.json()
pdbid=list(data.keys())[0]
data = data[pdbid]
assembly_data = data['assembly']

no_interfaces = assembly_data['interface_count']
dissociation_energy = assembly_data['dissociation_energy']

print('Assembly {} has {} interfaces'.format(pdbid,no_interfaces))
print('Assembly {} has a dissociation energy of {} kcal/mol'.format(pdbid,dissociation_energy))

Assembly 3gz1 has 3 interfaces
Assembly 3gz1 has a dissociation energy of 16.95 kcal/mol


## 2) Single interface in an assembly 

The second API endpoint provides a JSON response containing details of a single interface selected within assembly, based on a given PDB entry, an assembly ID and an interface ID. Descriptions of the interface parameters can be found at [Apiary](https://pisalite.docs.apiary.io/#reference/0/pisaqualifierjson/interaction-interface-data-per-pdb-assembly-entry).

For a single interface, the data contains a list of contacts or pair interactions and detailed properties such as atom site IDs, UniProt residue indices, residue IDs, chains, sequence IDs, distances, and more.

The API enpoint pattern is : *https://wwwdev.ebi.ac.uk/pdbe/api/pisa/interface/:pdbid/:assemblyid/:interfaceid*



For instance, from previous example we know that the assembly entry **pdbid = 3gz1** with **assemblyid = 1** has 3 interfaces. We can access each interface details with the interface ID, e.g. **interfaceid = 2** :

In [45]:
response_single_interface = requests.get("https://wwwdev.ebi.ac.uk/pdbe/api/pisa/interface/3gz1/1/2")

response_single_interface.json()

{'interface_id': '2',
 'interface_area': 603.16,
 'solvation_energy': -8.48,
 'stabilization_energy': -10.55,
 'p_value': 0.479,
 'number_interface_residues': 13,
 'number_hydrogen_bonds': 3,
 'number_covalent_bonds': 0,
 'number_disulfide_bonds': 0,
 'number_salt_bridges': 2,
 'number_other_bonds': 86,
 'hydrogen_bonds': {'bond_distances': [3.05, 2.79, 2.59],
  'atom_site_1_chains': ['B', 'B', 'B'],
  'atom_site_1_residues': ['GLN', 'TYR', 'TYR'],
  'atom_site_1_label_asym_ids': ['B', 'B', 'B'],
  'atom_site_1_orig_label_asym_ids': ['B', 'B', 'B'],
  'atom_site_1_unp_accs': ['P0A2U4', 'P0A2U4', 'P0A2U4'],
  'atom_site_1_unp_nums': ['112', '47', '40'],
  'atom_site_1_seq_nums': ['112', '47', '40'],
  'atom_site_1_label_seq_ids': ['112', '47', '40'],
  'atom_site_1_label_atom_ids': [' NE2', ' OH ', ' OH '],
  'atom_site_1_inscodes': [None, None, None],
  'atom_site_2_chains': ['P', 'P', 'P'],
  'atom_site_2_residues': ['ILE', 'PRO', 'LYS'],
  'atom_site_2_label_asym_ids': ['C', 'C', 'C'

With all the interface details available in this JSON response, we can e.g. extract information about the number of interface residues or number of contacts, for a selected type of contact (hydrogen bond, salt bridge, covalent bond, etc.)

In [46]:
interface_data = response_single_interface.json()

interface_id=interface_data.get('interface_id')
no_interface_residues=interface_data.get('number_interface_residues')
no_hydrogen_bonds = interface_data.get('number_hydrogen_bonds')
no_salt_bridges = interface_data.get('number_salt_bridges')

         
print("The interface {} of preferred assembly {} has {} interface residues with {} hydrogen bonds and {} salt bridges.".
      format(interface_id,pdbid,no_interface_residues,no_hydrogen_bonds, no_salt_bridges))

The interface 2 of preferred assembly 3gz1 has 13 interface residues with 3 hydrogen bonds and 2 salt bridges.


It is possible e.g. to read the label sequence and atom indexes for each hydrogen bond in the interface.

In [39]:
import tabulate
import pandas as pd
from pandas import DataFrame
from tabulate import tabulate

interface_pair_prop = []

for key, prop in interface_data.items():
    if key in [
            "hydrogen_bonds",
        ]:
            label_seq_atom_1 = prop["atom_site_1_label_seq_ids"]
            label_seq_atom_2 = prop["atom_site_2_label_seq_ids"]
            label_id_atom_1 = prop["atom_site_1_label_atom_ids"]
            label_id_atom_2 = prop["atom_site_2_label_atom_ids"]

for (item1,item2,item3,item4) in zip(
        label_seq_atom_1,label_id_atom_1,
        label_seq_atom_2,label_id_atom_2
    ):
        
        interface_pair_prop.append([item1,item2,item3,item4])

df = pd.DataFrame(
        interface_pair_prop, columns=
        [
            "label_sequence_index_a","label_atom_index_a",
            "label_sequence_index_b","label_atom_index_b"
            
        ],
    )

print (tabulate(df,headers='keys', tablefmt='psql'))


+----+--------------------------+----------------------+--------------------------+----------------------+
|    |   label_sequence_index_a | label_atom_index_a   |   label_sequence_index_b | label_atom_index_b   |
|----+--------------------------+----------------------+--------------------------+----------------------|
|  0 |                      112 | NE2                  |                       12 | O                    |
|  1 |                       47 | OH                   |                       15 | O                    |
|  2 |                       40 | OH                   |                       18 | O                    |
+----+--------------------------+----------------------+--------------------------+----------------------+


## 3) Every interface in an assembly

This API endpoint provides a JSON response containing details about all the interfaces within an assembly, based on the given PDB entry and assembly ID. You can find comprehensive descriptions of the interface parameters in the returned data at [Apiary](https://pisalite.docs.apiary.io/#reference/0/pisaqualifierjson/interaction-interface-data-per-pdb-assembly-entry).

The data encompasses both structural and thermodynamic information for the assembly (e.g., number of interfaces, size, dissociation energy, etc.) and interface data. The latter includes specifics for each enumerated interface in the assembly (e.g., interface area, stabilization energy, number of interface residues, etc.) as well as interface interactions (e.g., hydrogen bonds, salt bridges, covalent bonds, etc.).

For each interface, the data features a list of contacts or pair interactions, detailing properties such as atom site IDs, UniProt residue indices, residue IDs, chains, sequence IDs, distances, and more.

The API endpoint pattern is: *https://wwwdev.ebi.ac.uk/pdbe/api/pisa/interfaces/:pdbid/:assemblyid*


For example, for entry **pdbid = 3gz1** and preferred assembly **assemblyid = 1**, there are 3 interfaces identified:

In [13]:
response_all_interfaces = requests.get("https://wwwdev.ebi.ac.uk/pdbe/api/pisa/interfaces/3gz1/1")
print('There are %i interfaces' % response_all_interfaces.json()['3gz1']['assembly']['interface_count'])

There are 3 interfaces


The complete JSON can be viewed below:

In [40]:
response_all_interfaces.json()

{'3gz1': {'assembly_id': '1',
  'pisa_version': '2.0',
  'assembly': {'mmsize': '4',
   'dissociation_energy': 16.95,
   'accessible_surface_area': 15635.16,
   'buried_surface_area': 6031.94,
   'entropy': 19.65,
   'dissociation_area': 2034.9,
   'solvation_energy_gain': -35.09,
   'formula': 'A(2)BCa(3)',
   'composition': 'ABPQ[GOL](3)',
   'interface_count': 3,
   'interfaces': [{'interface_id': '1',
     'interface_area': 1357.57,
     'solvation_energy': -21.98,
     'stabilization_energy': -28.2,
     'p_value': 0.056,
     'number_interface_residues': 142,
     'number_hydrogen_bonds': 14,
     'number_covalent_bonds': 0,
     'number_disulfide_bonds': 0,
     'number_salt_bridges': 0,
     'number_other_bonds': 127,
     'hydrogen_bonds': {'bond_distances': [2.9,
       3.75,
       3.63,
       3.84,
       2.78,
       3.71,
       3.78,
       3.85,
       3.7,
       3.51,
       3.0,
       3.64,
       2.66,
       2.99],
      'atom_site_1_chains': ['A',
       'A',
  

With comprehensive interface details available in the JSON, it is possible to extract information such as the number of hydrogen bonds for each interface.:

In [41]:
assembly_data_all = response_all_interfaces.json()

pdbid=list(assembly_data_all.keys())[0]
assembly_data_all = assembly_data_all[pdbid]

assembly_data_all = assembly_data_all['assembly']
interfaces = assembly_data_all['interfaces']

for interface in interfaces:
    
    interface_id=interface.get('interface_id')
    no_hydrogen_bonds = interface.get('number_hydrogen_bonds')  
         
    print("The number of hydrogen bonds in interface {} is {}".format(interface_id,no_hydrogen_bonds))
    

The number of hydrogen bonds in interface 1 is 14
The number of hydrogen bonds in interface 2 is 3
The number of hydrogen bonds in interface 3 is 3


It is also possible to read the UniProt sequence indexes for each hydrogen bond in the interface.

In [42]:
import tabulate
import pandas as pd
from pandas import DataFrame
from tabulate import tabulate

interface_pairs = []

for interface in interfaces:
    
    interface_id=interface.get('interface_id')
    
    for key, prop in interface.items():
        if key in [
            "hydrogen_bonds",
        ]:
            unp_nums_atom_1 = prop["atom_site_1_unp_nums"]
            unp_nums_atom_2 = prop["atom_site_2_unp_nums"]

    for (item1,item2) in zip(unp_nums_atom_1,unp_nums_atom_2):
        
        interface_pairs.append([interface_id,item1,item2])

df = pd.DataFrame(
        interface_pairs, columns=
        [
            "interface_id","uniprot_residue_index_a","uniprot_residue_index_b"
        ],
    )

print (tabulate(df,headers='keys', tablefmt='psql'))


+----+----------------+---------------------------+---------------------------+
|    |   interface_id |   uniprot_residue_index_a |   uniprot_residue_index_b |
|----+----------------+---------------------------+---------------------------|
|  0 |              1 |                        12 |                        68 |
|  1 |              1 |                        24 |                        56 |
|  2 |              1 |                        24 |                        85 |
|  3 |              1 |                        87 |                        42 |
|  4 |              1 |                       133 |                        30 |
|  5 |              1 |                         8 |                        68 |
|  6 |              1 |                        18 |                        80 |
|  7 |              1 |                        19 |                        60 |
|  8 |              1 |                        19 |                        80 |
|  9 |              1 |                 